# Riddler Classic
After a long night of frivolous quackery, two delirious ducks are having a difficult time finding each other in their pond. The pond happens to contain a 3×3 grid of rocks.

Every minute, each duck randomly swims, independently of the other duck, from one rock to a neighboring rock in the 3×3 grid — up, down, left or right, but not diagonally. So if a duck is at the middle rock, it will next swim to one of the four side rocks with probability 1/4. From a side rock, it will swim to one of the two adjacent corner rocks or back to the middle rock, each with probability 1/3. And from a corner rock, it will swim to one of the two adjacent side rocks with probability 1/2.

If the ducks both start at the middle rock, then on average, how long will it take until they’re at the same rock again? (Of course, there’s a 1/4 chance that they’ll swim in the same direction after the first minute, in which case it would only take one minute for them to be at the same rock again. But it could take much longer, if they happen to keep missing each other.)

Extra credit: What if there are three or more ducks? If they all start in the middle rock, on average, how long will it take until they are all at the same rock again?

In [148]:
import collections
import itertools
import numpy as np
import toolz
from typing import Iterator, List, Tuple

In [149]:
Position = collections.namedtuple('position', ["x", "y"])
class Duck(object):
    def __init__(self, N:int):
        """Initialize a duck.
        
        Args:
          N: number of grid points.
        """
        self.N = N # N represents the size of the grid
        # Start at the middle
        if N%2 == 1:
            pos = int((N + 1) / 2)
        else:
            # start randomly at either of the locations
            rand = np.random.random()
            if rand < 0.5:
                pos = int(N / 2)
            else:
                pos= int(N / 2) + 1
        self.position = Position(pos, pos)
    
    def _get_next_locations(self) ->List[Position]:
        """Calculates the next location"""
        directions = []
        x, y = self.position
        # See the up direction
        _ = (directions.append(Position(x, y-1)) 
             if y - 1 >= 1 else None)
        # See the down direction
        _ = (directions.append(Position(x, y+1)) 
             if y + 1 <= self.N else None)
        # see the left direction
        _ = (directions.append(Position(x-1, y)) 
             if x - 1 >= 1 else None)
        # see the right direction
        _ = (directions.append(Position(x+1, y)) 
             if x + 1 <= self.N else None)
        return directions
    
    def _set_next_location(self) -> Position:
        """Gets the next move. Also updates the state of the duck.
        
        Returns:
          A tuple of the next direction.
        """
        directions = self._get_next_locations()
        index = np.random.randint(len(directions))
        self.position = directions[index]
        return self.position
    
    def moves(self) -> Iterator[Position]:
        """Returns an infinite iterator of next moves."""
        while True:
            yield self._set_next_location()

In [150]:
def is_same_for_all_ducks(positions: Tuple[Position]) -> bool:
    """Returns True if the position for all ducks are the same."""
    return (len(set(map(lambda pos: pos.x, positions))) == 1 and 
            len(set(map(lambda pos: pos.y, positions))) == 1)

def is_different_for_all_ducks(positions: Tuple[Position]) -> bool:
    return not is_same_for_all_ducks(positions)

def find_num_minutes_before_meet(num_ducks: int, size_grid: int) -> int:
    """Returns the number of minutes before all ducks meet first.
    
    Args:
      num_ducks: Number of ducks in the system
      size_grid: The size of the pond.
      
    Returns:
      Number of minutes it takes for the ducks to meet.
    """
    all_positions = zip(*[Duck(size_grid).moves() 
                          for _ in range(num_ducks)])
    till_first_meet = itertools.takewhile(is_different_for_all_ducks,
                        all_positions)
    # If we meet at the very first time then we get an empty iterator.
    return max(1, sum(1 for _ in till_first_meet)) 

In [151]:
def find_mean_num_minutes_before_meet(num_ducks: int,
                                      size_grid: int, 
                                      num_sims: int) -> float:
    return np.mean([find_num_minutes_before_meet(num_ducks, 
                                                 size_grid) 
                    for _ in range(num_sims)])

In [152]:
(find_mean_num_minutes_before_meet(2, 3, 100000))

4.16098

In [154]:
# Lets plot what happnes as the number of ducks increase
means = [find_mean_num_minutes_before_meet(num_ducks, 3, 1000) for num_ducks in range(2, 7)]
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(range(2, 7), means)
ax.set_ylabel('Mean minutes to meet')
ax.set_xlabel('Number of ducks')

Matplotlib is building the font cache using fc-list. This may take a moment.


Text(0.5, 0, 'Number of ducks')

In [155]:
plt.show()

In [156]:
means

[3.853, 19.318, 65.645, 237.51, 821.02]